In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv
/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv
/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv


# Cleaning Data

In [2]:
train_data = pd.read_csv("/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv")
test_data = pd.read_csv("/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv")

train_data.head(20)

,Country Code,Country Name,Indicator,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015]
0,Afg,Afghanistan,Access to electricity (% of population),4.446890831,9.294527054,14.13361645,18.9711647,23.81418228,28.66967201,33.54441833,38.44000244,42.4,48.27900696,42.7,43.22201891,69.1,68.0408783,89.5,71.5
1,Afg,Afghanistan,Agricultural land (% of land area),57.94581666,57.94734986,57.93968385,58.08380479,58.15126566,58.13440044,58.12366803,58.12980084,58.13286724,58.13286724,58.13440044,58.13133404,58.12980084,58.12366803,58.12366803,58.12366803
2,Afg,Afghanistan,"Annual freshwater withdrawals, total (% of int...",43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668
3,Afg,Afghanistan,Arable land (% of land area),11.77958696,11.77958696,11.77192095,11.91604189,11.98350275,11.96663754,11.94977232,11.94977232,11.94977232,11.94823912,11.94823912,11.94517272,11.94363951,11.93597351,11.91450869,11.90530948
4,Afg,Afghanistan,Forest area (% of land area),1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994
5,Afg,Afghanistan,Electric power consumption (kWh per capita),..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..
6,Afg,Afghanistan,Energy use (kg of oil equivalent per capita),..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..
7,Afg,Afghanistan,Renewable electricity output (% of total elect...,74.98909408,72.81145999,79.0639712,70.24972856,70.89084065,74.06181015,70.75575027,72,68.65482234,87.17660292,85.98654709,82.48756219,85.90998043,78.6364081,85.32354903,86.05011125
8,Afg,Afghanistan,Renewable energy consumption (% of total final...,44.99,45.6,37.83,36.66,44.24,33.88,31.89,28.78,21.17,16.53,15.15,12.61,15.36,16.86,18.93,17.53
9,Afg,Afghanistan,Population growth (annual %),1.443803024,0.742516834,6.449321478,7.541018965,3.933177688,3.576508004,4.139678035,1.793195727,2.002333262,3.561288374,2.894904104,3.689508302,4.077627729,3.466788304,3.657576065,3.121341229


# For Submission File

In [3]:
test_ids = test_data["Country Name"]
#yearCols = train_data.columns[train_data.columns.str.contains('YR')]
print(test_ids)

0       Afghanistan
1       Afghanistan
2       Afghanistan
3       Afghanistan
4       Afghanistan
           ...     
2921          World
2922          World
2923          World
2924          World
2925          World
Name: Country Name, Length: 2926, dtype: object


# Cleaning Unnecessary Data

In [4]:
def clean(data):
    
    data.replace('..', np.nan, inplace=True) 

    numeric_cols = data.columns[data.columns.str.contains('YR')]
    data[numeric_cols] = data[numeric_cols].apply(pd.to_numeric, errors='coerce')

    for col in numeric_cols:
        data[col] = data[col].fillna(data[col].median())

    return data

In [5]:
train_data = clean(train_data)
test_data = clean(test_data)

# Encoding

In [6]:
from sklearn import preprocessing
labelEncoder = preprocessing.LabelEncoder()

encode_cols = ["Country Name", "Indicator"]
for col in encode_cols:
    train_data[col] = labelEncoder.fit_transform(train_data[col])
    test_data[col] = labelEncoder.transform(test_data[col])
    #print(labelEncoder.classes_)

train_data.head(30)

,Country Code,Country Name,Indicator,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015]
0,Afg,0,0,4.446891,9.294527,14.133616,18.971165,23.814182,28.669672,33.544418,38.440002,42.400000,48.279007,42.700000,43.222019,69.100000,68.040878,89.500000,71.500000
1,Afg,0,1,57.945817,57.947350,57.939684,58.083805,58.151266,58.134400,58.123668,58.129801,58.132867,58.132867,58.134400,58.131334,58.129801,58.123668,58.123668,58.123668
2,Afg,0,2,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907
3,Afg,0,3,11.779587,11.779587,11.771921,11.916042,11.983503,11.966638,11.949772,11.949772,11.949772,11.948239,11.948239,11.945173,11.943640,11.935974,11.914509,11.905309
4,Afg,0,7,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782
5,Afg,0,5,29.576545,29.333333,29.398370,29.333333,30.100000,30.250946,30.142328,30.031714,29.850000,30.009543,30.332978,30.224755,30.229220,30.644329,29.625668,21.342090
6,Afg,0,6,29.576545,29.333333,29.398370,29.333333,30.100000,30.250946,30.142328,30.031714,29.850000,30.009543,30.332978,30.224755,30.229220,30.644329,29.625668,21.342090
7,Afg,0,10,74.989094,72.811460,79.063971,70.249729,70.890841,74.061810,70.755750,72.000000,68.654822,87.176603,85.986547,82.487562,85.909980,78.636408,85.323549,86.050111
8,Afg,0,11,44.990000,45.600000,37.830000,36.660000,44.240000,33.880000,31.890000,28.780000,21.170000,16.530000,15.150000,12.610000,15.360000,16.860000,18.930000,17.530000
9,Afg,0,9,1.443803,0.742517,6.449321,7.541019,3.933178,3.576508,4.139678,1.793196,2.002333,3.561288,2.894904,3.689508,4.077628,3.466788,3.657576,3.121341


# Making year columns integers

In [7]:
def convert_year_to_int(df,start):
    
    indicator_columns = list(df.columns[:start])

    for column in df.columns[start:]:
        year = int(column.split(' ')[0])
        indicator_columns.append(year)

    df.columns = indicator_columns
    return df 

In [8]:

train_year_start = 3 
test_year_start = 2

train_data = convert_year_to_int(train_data,train_year_start)
test_data = convert_year_to_int(test_data,test_year_start)

In [9]:
train_data.head(50)

,Country Code,Country Name,Indicator,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Afg,0,0,4.446891,9.294527,14.133616,18.971165,23.814182,28.669672,33.544418,38.440002,42.400000,48.279007,42.700000,43.222019,69.100000,68.040878,89.500000,71.500000
1,Afg,0,1,57.945817,57.947350,57.939684,58.083805,58.151266,58.134400,58.123668,58.129801,58.132867,58.132867,58.134400,58.131334,58.129801,58.123668,58.123668,58.123668
2,Afg,0,2,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907
3,Afg,0,3,11.779587,11.779587,11.771921,11.916042,11.983503,11.966638,11.949772,11.949772,11.949772,11.948239,11.948239,11.945173,11.943640,11.935974,11.914509,11.905309
4,Afg,0,7,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782
5,Afg,0,5,29.576545,29.333333,29.398370,29.333333,30.100000,30.250946,30.142328,30.031714,29.850000,30.009543,30.332978,30.224755,30.229220,30.644329,29.625668,21.342090
6,Afg,0,6,29.576545,29.333333,29.398370,29.333333,30.100000,30.250946,30.142328,30.031714,29.850000,30.009543,30.332978,30.224755,30.229220,30.644329,29.625668,21.342090
7,Afg,0,10,74.989094,72.811460,79.063971,70.249729,70.890841,74.061810,70.755750,72.000000,68.654822,87.176603,85.986547,82.487562,85.909980,78.636408,85.323549,86.050111
8,Afg,0,11,44.990000,45.600000,37.830000,36.660000,44.240000,33.880000,31.890000,28.780000,21.170000,16.530000,15.150000,12.610000,15.360000,16.860000,18.930000,17.530000
9,Afg,0,9,1.443803,0.742517,6.449321,7.541019,3.933178,3.576508,4.139678,1.793196,2.002333,3.561288,2.894904,3.689508,4.077628,3.466788,3.657576,3.121341


In [10]:
test_data.head(50)

,Country Name,Indicator,2016,2017,2018,2019,2020
0,0,0,97.700000,97.700000,93.430878,97.700000,97.700000
1,0,1,58.123668,58.123668,58.276988,58.276988,58.741548
2,0,2,43.015907,43.015907,43.015907,43.015907,43.015907
3,0,3,11.850114,11.804118,11.949772,11.940573,12.003434
4,0,7,1.852782,1.852782,1.852782,1.852782,1.852782
5,0,5,27.748777,27.608983,27.604467,27.975597,28.083025
6,0,6,27.748777,27.608983,27.604467,27.975597,28.083025
7,0,10,27.748777,27.608983,27.604467,27.975597,28.083025
8,0,11,19.920000,19.210000,17.960000,18.510000,17.580000
9,0,9,2.581549,2.866492,2.885208,2.908529,3.134747


In [11]:
train_data.head()

,Country Code,Country Name,Indicator,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Afg,0,0,4.446891,9.294527,14.133616,18.971165,23.814182,28.669672,33.544418,38.440002,42.400000,48.279007,42.700000,43.222019,69.100000,68.040878,89.500000,71.500000
1,Afg,0,1,57.945817,57.947350,57.939684,58.083805,58.151266,58.134400,58.123668,58.129801,58.132867,58.132867,58.134400,58.131334,58.129801,58.123668,58.123668,58.123668
2,Afg,0,2,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907,43.015907
3,Afg,0,3,11.779587,11.779587,11.771921,11.916042,11.983503,11.966638,11.949772,11.949772,11.949772,11.948239,11.948239,11.945173,11.943640,11.935974,11.914509,11.905309
4,Afg,0,7,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782,1.852782


# Reshaping the dataframe to find x_train,y_train


In [12]:
def create_training_data_util(train_data,country_name,year,test=False):
    
    country_frame = train_data[train_data['Country Name']==country_name]
    
    indicators = [country_name]
    
    indicators += country_frame[year].tolist()
    
    y_train = [indicators[-1]]
    
    x_train = indicators[0:-1]
    
    if(test):
        x_train = indicators
    

    return x_train,y_train

In [13]:
def create_training_data(train_data,start,test):
    
    countries = train_data['Country Name'].unique()
    years = train_data.columns[start:].tolist()
    
    x_train = []
    y_train = []
    
    for country in countries:
        
        for year in years:
            
            u_train,v_train = create_training_data_util(train_data,country,year,test)
            x_train.append(u_train)
            y_train.append(v_train)
    
    x_train = pd.DataFrame(x_train)
    y_train = pd.DataFrame(y_train)
    
    return x_train,y_train

In [14]:
x_train, y_train = create_training_data(train_data,train_year_start,False)

x_train.head(50)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,4.446891,57.945817,43.015907,11.779587,1.852782,29.576545,29.576545,74.989094,44.99,1.443803,29.576545
1,0,9.294527,57.947350,43.015907,11.779587,1.852782,29.333333,29.333333,72.811460,45.60,0.742517,29.333333
2,0,14.133616,57.939684,43.015907,11.771921,1.852782,29.398370,29.398370,79.063971,37.83,6.449321,182.174037
3,0,18.971165,58.083805,43.015907,11.916042,1.852782,29.333333,29.333333,70.249729,36.66,7.541019,199.643228
4,0,23.814182,58.151266,43.015907,11.983503,1.852782,30.100000,30.100000,70.890841,44.24,3.933178,221.830531
5,0,28.669672,58.134400,43.015907,11.966638,1.852782,30.250946,30.250946,74.061810,33.88,3.576508,254.115274
6,0,33.544418,58.123668,43.015907,11.949772,1.852782,30.142328,30.142328,70.755750,31.89,4.139678,274.015394
7,0,38.440002,58.129801,43.015907,11.949772,1.852782,30.031714,30.031714,72.000000,28.78,1.793196,376.318296
8,0,42.400000,58.132867,43.015907,11.949772,1.852782,29.850000,29.850000,68.654822,21.17,2.002333,382.533804
9,0,48.279007,58.132867,43.015907,11.948239,1.852782,30.009543,30.009543,87.176603,16.53,3.561288,453.387382


We don't need the country name, so resetting x_train excluding column 1

In [15]:
x_train = x_train.iloc[:,1:]

In [16]:
x_train.head()

,1,2,3,4,5,6,7,8,9,10,11
0,4.446891,57.945817,43.015907,11.779587,1.852782,29.576545,29.576545,74.989094,44.99,1.443803,29.576545
1,9.294527,57.947350,43.015907,11.779587,1.852782,29.333333,29.333333,72.811460,45.60,0.742517,29.333333
2,14.133616,57.939684,43.015907,11.771921,1.852782,29.398370,29.398370,79.063971,37.83,6.449321,182.174037
3,18.971165,58.083805,43.015907,11.916042,1.852782,29.333333,29.333333,70.249729,36.66,7.541019,199.643228
4,23.814182,58.151266,43.015907,11.983503,1.852782,30.100000,30.100000,70.890841,44.24,3.933178,221.830531


# Using standard scaler to fit and transform
It is used to scale down the values that might act as outliers

In [17]:
from sklearn.preprocessing import StandardScaler

def scale_dataset(df):
    
    sc = StandardScaler()
    scaled_dataset = sc.fit_transform(df)
    # it will return an array
    # converting to a dataframe again
    df = pd.DataFrame(scaled_dataset)
    
    return df

In [18]:
scaled_x_train =  scale_dataset(x_train)
scaled_x_train.head()

,0,1,2,3,4,5,6,7,8,9,10
0,-2.410694,1.042157,-0.093927,-0.179472,-1.330409,-0.543451,-0.625396,1.526154,0.497384,-0.037423,-0.603952
1,-2.250614,1.042233,-0.093927,-0.179472,-1.330409,-0.543506,-0.625491,1.455214,0.518373,-0.342169,-0.603964
2,-2.090817,1.041852,-0.093927,-0.180066,-1.330409,-0.543491,-0.625465,1.658900,0.251028,2.137741,-0.596643
3,-1.931071,1.049019,-0.093927,-0.168889,-1.330409,-0.543506,-0.625491,1.371761,0.210771,2.612142,-0.595806
4,-1.771144,1.052373,-0.093927,-0.163658,-1.330409,-0.543331,-0.625190,1.392646,0.471579,1.044343,-0.594743


# Shuffling the dataset

In [19]:
scaled_train_set = pd.concat([scaled_x_train,y_train],axis=1)

shuffled_train_set = scaled_train_set.sample(frac=1,random_state=42)

# Splitting shuffled data into train_set and test_set

In [20]:
from sklearn.model_selection import train_test_split


train_set,test_set = train_test_split(shuffled_train_set,test_size=0.3,random_state=42)

# Recovering x_train,y_train,x_test,y_test

In [21]:
x_train = train_set.iloc[:,0:-1]
y_train = train_set.iloc[:,-1]
x_test = test_set.iloc[:,0:-1]
y_test = test_set.iloc[:,-1]

# Time to apply models

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Linear Regression

In [23]:
model = LinearRegression()
model.fit(x_train,y_train)
predictions = model.predict(x_test)
mean_squared_error(y_test,predictions)

44.13928621143912

# Ridge Regression

In [24]:
model = Ridge(alpha=1.0)  # regularization strength
model.fit(x_train,y_train)
predictions = model.predict(x_test)
mean_squared_error(y_test,predictions)

44.136383832241094

# Lasso Regression

In [25]:
model = Lasso(alpha=1.0)  # regularization strength
model.fit(x_train,y_train)
predictions = model.predict(x_test)
mean_squared_error(y_test,predictions)

46.61669971171827

# Decision Trees

In [26]:
from sklearn.tree import DecisionTreeRegressor

# Initialize the model
model = DecisionTreeRegressor()

# Train the model
model.fit(x_train, y_train)

# Make predictions
predictions = model.predict(x_test)

# Evaluate the model
mean_squared_error(y_test, predictions)

6.256444172159494

# Random Forest Regressor

In [27]:
# Initialize RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)  # Example number of trees

# Train the model
model.fit(x_train, y_train.values.ravel())

# Make predictions
predictions = model.predict(x_test)

# Evaluate the model
mean_squared_error(y_test, predictions)

3.033965465850128

# Hypertuning Forest Regressor

In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel

rf = RandomForestRegressor(random_state=42)

# Define hyperparameters to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Perform grid search
grid_search.fit(x_train, y_train.values.ravel())

# Get best hyperparameters
best_params = grid_search.best_params_

# Initialize RandomForestRegressor with best hyperparameters
best_rf = RandomForestRegressor(**best_params, random_state=42)

# Train the model
best_rf.fit(x_train, y_train.values.ravel())

# Make predictions
predictions = best_rf.predict(x_test)

# Evaluate the model
mean_squared_error(y_test, predictions)

1.7211698380126874

# Using the function to extract final_x from test_data

**final_y will be null and final_x will be the input to our final model**

In [29]:
final_x,final_y = create_training_data(test_data,test_year_start,True)
# final_x must be scaled dow and it's first column must be dropped 
final_x.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,97.700000,58.123668,43.015907,11.850114,1.852782,27.748777,27.748777,27.748777,19.92,2.581549,523.053012
1,0,97.700000,58.123668,43.015907,11.804118,1.852782,27.608983,27.608983,27.608983,19.21,2.866492,526.140801
2,0,93.430878,58.276988,43.015907,11.949772,1.852782,27.604467,27.604467,27.604467,17.96,2.885208,492.090632
3,0,97.700000,58.276988,43.015907,11.940573,1.852782,27.975597,27.975597,27.975597,18.51,2.908529,497.741429
4,0,97.700000,58.741548,43.015907,12.003434,1.852782,28.083025,28.083025,28.083025,17.58,3.134747,512.055098
